## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data (use this if running your Jupyter notebook within Coursera)
df = pd.read_csv("nap_no_nap.csv") 

In [0]:
# Import the data (uncomment the line below and use this if you downloaded the Jupyter notebook)
# df = pd.read_csv("https://raw.githubusercontent.com/UMstatspy/UMStatsPy/master/Course_2/nap_no_nap.csv")


In [3]:
# First, look at the DataFrame to get a sense of the data
df

id     sex  age (months)  dlmo time  days napped  napping  \
0    1  female          33.7      19.24            0        0   
1    2  female          31.5      18.27            0        0   
2    3    male          31.9      19.14            0        0   
3    4  female          31.6      19.69            0        0   
4    5  female          33.0      19.52            0        0   
5    6  female          36.2      18.22            4        1   
6    7    male          36.3      19.28            1        1   
7    8    male          30.0      21.06            5        1   
8    9    male          33.2      19.38            2        1   
9   10  female          37.1      19.93            3        1   
10  11    male          32.9      18.79            4        1   
11  12  female          35.0      19.65            5        1   
12  13    male          35.1      19.83            3        1   
13  14  female          35.6      19.88            4        1   
14  15  female          36.6      19.94            4        1   
15  16    male          36.5      20.25            3        1   
16  17  female          33.7      20.33            5        1   
17  18    male          36.4      20.16            5        1   
18  19  female          33.6      19.68            3        1   
19  20    male          33.8      20.51            3        1   

    nap lights outl time  nap sleep onset  nap midsleep  nap sleep offset  \
0                    NaN              NaN           NaN               NaN   
1                    NaN              NaN           NaN               NaN   
2                    NaN              NaN           NaN               NaN   
3                    NaN              NaN           NaN               NaN   
4                    NaN              NaN           NaN               NaN   
5                  14.00            14.22         15.00             15.78   
6                  14.75            15.03         15.92             16.80   
7                  13.09            13.43         14.44             15.46   
8                  14.41            14.42         15.71             17.01   
9                  13.12            13.42         14.31             15.19   
10                 13.99            14.03         14.85             15.68   
11                 13.18            13.45         14.33             15.21   
12                 13.94            14.48         15.26             16.03   
13                 12.68            13.08         13.92             14.76   
14                 12.71            12.88         13.80             14.72   
15                 13.74            14.68         15.66             16.64   
16                 13.15            13.87         14.49             15.11   
17                 12.47            12.56         13.30             14.05   
18                 14.71            14.85         15.46             16.07   
19                 12.68            13.54         14.30             15.07   

    nap wake time  nap duration  nap time in bed  night bedtime  \
0             NaN           NaN              NaN          20.45   
1             NaN           NaN              NaN          19.23   
2             NaN           NaN              NaN          19.60   
3             NaN           NaN              NaN          19.46   
4             NaN           NaN              NaN          19.21   
5           16.28         93.75           137.00          19.95   
6           16.08        106.00            80.00          20.60   
7           15.82        121.60           163.80          22.01   
8           16.60        155.50           131.25          20.24   
9           15.30        106.67           130.67          20.78   
10          16.10         98.75           126.60          19.45   
11          15.35        105.80           130.40          20.18   
12          15.78         93.33           110.20          20.22   
13          15.00        100.75           139.33          20.26   
14          14.

**Question**: What variable is used in the column 'napping' to indicate a toddler takes a nap?  
**Question**: What is the sample size $n$? What is the sample size for toddlers who nap, $n_1$, and toddlers who don't nap, $n_2$?

### Average bedtime confidence interval for napping and non napping toddlers
Create two 95% confidence intervals for the average bedtime, one for toddler who nap and one for toddlers who don't.

Before any analysis, we will convert 'night bedtime' into decimalized time.

In [4]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [5]:
napping = df[df["napping"] == 1]
napping

id     sex  age (months)  dlmo time  days napped  napping  \
5    6  female          36.2      18.22            4        1   
6    7    male          36.3      19.28            1        1   
7    8    male          30.0      21.06            5        1   
8    9    male          33.2      19.38            2        1   
9   10  female          37.1      19.93            3        1   
10  11    male          32.9      18.79            4        1   
11  12  female          35.0      19.65            5        1   
12  13    male          35.1      19.83            3        1   
13  14  female          35.6      19.88            4        1   
14  15  female          36.6      19.94            4        1   
15  16    male          36.5      20.25            3        1   
16  17  female          33.7      20.33            5        1   
17  18    male          36.4      20.16            5        1   
18  19  female          33.6      19.68            3        1   
19  20    male          33.8      20.51            3        1   

    nap lights outl time  nap sleep onset  nap midsleep  nap sleep offset  \
5                  14.00            14.22         15.00             15.78   
6                  14.75            15.03         15.92             16.80   
7                  13.09            13.43         14.44             15.46   
8                  14.41            14.42         15.71             17.01   
9                  13.12            13.42         14.31             15.19   
10                 13.99            14.03         14.85             15.68   
11                 13.18            13.45         14.33             15.21   
12                 13.94            14.48         15.26             16.03   
13                 12.68            13.08         13.92             14.76   
14                 12.71            12.88         13.80             14.72   
15                 13.74            14.68         15.66             16.64   
16                 13.15            13.87         14.49             15.11   
17                 12.47            12.56         13.30             14.05   
18                 14.71            14.85         15.46             16.07   
19                 12.68            13.54         14.30             15.07   

    nap wake time  nap duration  nap time in bed  night bedtime  \
5           16.28         93.75           137.00         1235.0   
6           16.08        106.00            80.00         1260.0   
7           15.82        121.60           163.80         1321.0   
8           16.60        155.50           131.25         1224.0   
9           15.30        106.67           130.67         1278.0   
10          16.10         98.75           126.60         1185.0   
11          15.35        105.80           130.40         1218.0   
12          15.78         93.33           110.20         1222.0   
13          15.00        100.75           139.33         1226.0   
14          14.88        110.75           130.00         1228.0   
15          16.45        117.33           162.75         1246.0   
16          15.40         74.20           135.00         1243.0   
17          14.25         89.80           107.00         1202.0   
18          16.20         73.00            89.40         1190.0   
19          15.23         91.67           152.67         1218.0   

    night sleep onset  sleep onset latency  night midsleep time  \
5               20.25                 0.29                 1.26   
6               20.96                 0.36                 2.12   
7               22.53                 0.51                 2.92   
8               20.37                 0.13                 1.60   
9               21.63                 0.84                 2.20   
10              19.88                 0.44                 1.34   
11              20.84                 0.66                 1.93   
12              20.89                 0.67                 1.99   
13              20.80                 0.54            

In [6]:
bedtime_nap = napping["night bedtime"]
bedtime_nap

5     1235.0
6     1260.0
7     1321.0
8     1224.0
9     1278.0
10    1185.0
11    1218.0
12    1222.0
13    1226.0
14    1228.0
15    1246.0
16    1243.0
17    1202.0
18    1190.0
19    1218.0
Name: night bedtime, dtype: float64

In [8]:
notnapping = df[df["napping"] == 0]
notnapping

id     sex  age (months)  dlmo time  days napped  napping  \
0   1  female          33.7      19.24            0        0   
1   2  female          31.5      18.27            0        0   
2   3    male          31.9      19.14            0        0   
3   4  female          31.6      19.69            0        0   
4   5  female          33.0      19.52            0        0   

   nap lights outl time  nap sleep onset  nap midsleep  nap sleep offset  \
0                   NaN              NaN           NaN               NaN   
1                   NaN              NaN           NaN               NaN   
2                   NaN              NaN           NaN               NaN   
3                   NaN              NaN           NaN               NaN   
4                   NaN              NaN           NaN               NaN   

   nap wake time  nap duration  nap time in bed  night bedtime  \
0            NaN           NaN              NaN         1245.0   
1            NaN           NaN              NaN         1163.0   
2            NaN           NaN              NaN         1200.0   
3            NaN           NaN              NaN         1186.0   
4            NaN           NaN              NaN         1161.0   

   night sleep onset  sleep onset latency  night midsleep time  \
0              20.68                 0.23                 1.92   
1              19.48                 0.25                 1.09   
2              20.05                 0.45                 1.29   
3              19.50                 0.05                 1.89   
4              19.65                 0.45                 1.30   

   night wake time  night sleep duration  night time in bed  \
0             7.17                 629.4              643.0   
1             6.69                 672.4              700.4   
2             6.53                 628.8              682.6   
3             8.28                 766.6              784.0   
4             6.95                 678.0              718.0   

   24 h sleep duration  bedtime phase difference  \
0                629.4                     -1.21   
1                672.4                     -0.96   
2                628.8                     -0.46   
3                766.6                      0.23   
4                678.0                      0.31   

   sleep onset phase difference  midsleep phase difference  \
0                         -1.44                       6.68   
1                         -1.21                       6.82   
2                         -0.91                       6.15   
3                          0.19                       6.20   
4                         -0.13                       5.78   

   wake time phase difference  
0                       11.93  
1                       12.42  
2                       11.39  
3                       12.59  
4                       11.43

In [9]:
bedtime_no_nap = notnapping["night bedtime"]
bedtime_no_nap

0    1245.0
1    1163.0
2    1200.0
3    1186.0
4    1161.0
Name: night bedtime, dtype: float64

Now find the sample mean bedtime for nap and no_nap.

In [10]:
nap_mean_bedtime = np.mean(bedtime_nap)
nap_mean_bedtime

1233.0666666666666

In [11]:
no_nap_mean_bedtime = np.mean(bedtime_no_nap)
no_nap_mean_bedtime

1191.0

Now find the standard error for $\bar{X}_{nap}$ and $\bar{X}_{no\ nap}$.

In [12]:
nap_s_bedtime = np.std(bedtime_nap,ddof=1)
nap_s_bedtime

34.445540177143954

In [13]:
no_nap_s_bedtime = np.std(bedtime_no_nap, ddof=1)
no_nap_s_bedtime

34.30014577228499

**Question**: Given our sample sizes of $n_1$ and $n_2$ for napping and non napping toddlers respectively, how many degrees of freedom ($df$) are there for the associated $t$ distributions?

To build a 95% confidence interval, what is the value of t\*?  You can find this value using the percent point function: 
```
from scipy.stats import t

t.ppf(probabiliy, df)
```
This will return the quantile value such that to the left of this value, the tail probabiliy is equal to the input probabiliy (for the specified degrees of freedom). 

Example: to find the $t^*$ for a 90% confidence interval, we want $t^*$ such that 90% of the density of the $t$ distribution lies between $-t^*$ and $t^*$.

Or in other words if $X \sim t(df)$:

P($-t^*$ < X < $t^*$) = .90

Which, because the $t$ distribution is symmetric, is equivalent to finding $t^*$ such that:  

P(X < $t^*$) = .95

So the $t^*$ for a 90% confidence interval, and lets say df=10, will be:

t_star = t.ppf(.95, df=10)


In [0]:
# Find the t_stars for the 95% confidence intervals
nap_t_star = 

In [0]:
no_nap_t_star = 

**Quesion**: What is $t^*$ for nap and no nap?

Now to create our confidence intervals. For the average bedtime for nap and no nap, find the upper and lower bounds for the respective confidence intervals.

**Question**: What are the 95% confidence intervals, rounded to the nearest ten, for the average bedtime (in decimalized time) for toddlers who nap and for toddlers who don't nap? 

CI = $\bar{X} \pm \ t^* \cdot s.e.(\bar{X})$

**Challenge problem**: Write a function that inputs the column containing the data you want to build your confidence interval from and returns the confidence interval as a list or tuple (i.e. \[upper, lower\] or (upper, lower)).